In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [17]:
# read in cleaned train and test data; concatenate to single df
final_train_df = pd.read_csv('Data/train.csv')
final_test_df = pd.read_csv('Data/test.csv')
cl_df = pd.concat([final_train_df, final_test_df], ignore_index=True)

In [18]:
cl_df.labels.value_counts()


1    1043940
0     262416
Name: labels, dtype: int64

divide group into default-1 and non default-0  for bootstrapping


### Bootstrap Hypothesis Testing

In [19]:
np.random.seed(48)
N_rep = 10000

def draw_bs_reps(data, func, size=1):
    # initialize array
    bs_replicates = np.empty(size)
    
    # generate replicates
    for i in range(size):
        sample = np.random.choice(data, len(data))
        bs_replicates[i] = func(sample)
        
    return bs_replicates


In [20]:
# subset into arrays: charged_off(0) and fully_paid(1)

# charged_off = np.array(cl_df.annual_inc[cl_df.labels==0])
# fully_paid = np.array(cl_df.annual_inc[cl_df.labels==1])

charged_off = cl_df[cl_df.labels == 0]['annual_inc']
fully_paid = cl_df[cl_df.labels == 1]['annual_inc']


# ax = sns.distplot(charged_off, bins=int(np.sqrt(len(charged_off))), 
#                   kde=True, hist = False, label = 'defaulted')
# ax = sns.distplot(fully_paid, bins=int(np.sqrt(len(fully_paid))), 
#                   kde=True, hist = False , label = 'not defaulted')
# ax.set(xlabel='annual_inc', ylabel='probability', xlim = (0,5),
#        title = 'Age PDF for all, defaulted and not defaulted')

# plt.show()


In [21]:
# difference in observed mean for charged_off(0) and fully_paid(1): obs_diff_mean
obs_diff_mean = np.mean(charged_off) - np.mean(fully_paid)

In [22]:
# Calculate the 95% confidence interval for the difference between the means
charged_mean_replicates = draw_bs_reps(charged_off, np.mean, size=10000)
paid_mean_replicates = draw_bs_reps(fully_paid, np.mean, size=10000)

bs_diff_mean = draw_bs_reps(charged_off, np.mean, size=10000) - draw_bs_reps(fully_paid, np.mean, size=10000)

ci_bs_mean = np.percentile(bs_diff_mean, [2.5, 97.5])

print("The 95% confidence interval for the difference between replicant charged off and fully paid accounts is: ", ci_bs_mean)

The 95% confidence interval for the difference between replicant charged off and fully paid accounts is:  [-0.16053837 -0.15247379]


#### Bootstrap Procedure Using Mean As Test Statistic

In [23]:
# Bootstrap Procedure Using Mean As Test Statistic

# Merge the two observed samples into one sample of (n + m) observations
# Draw a bootstrap sample of (n + m) observations with replacement from the merged sample.
reps = 10000

# Concatenate the data sets: data
data = np.concatenate((charged_off, fully_paid))
charged_bs_rep_means = np.empty(reps)
paid_bs_rep_means = np.empty(reps)
        
# generate replicates
for i in range(reps):

    # Sample with replacement the concatenated array: sampled_data
    sampled_data = np.random.choice(data, len(data))

    # Split the sampled array into two: sample_1, sample_2
    charged_sample = sampled_data[:len(charged_off)]
    paid_sample = sampled_data[len(charged_off):]

    # Calculate means
    charged_bs_rep_means[i] = np.mean(charged_sample)
    paid_bs_rep_means[i] = np.mean(paid_sample)
#########################    
# Calculate difference in means: diff_bs_rep_means
diff_bs_rep_means = charged_bs_rep_means - paid_bs_rep_means

# Compute p-value: p
p = np.sum(diff_bs_rep_means >= obs_diff_mean) / len(diff_bs_rep_means)

# Print the result
print('p-value =', p)


p-value = 1.0


In [24]:
####################################
####################################
####################################

WHY IS P=1???!!

SyntaxError: invalid syntax (<ipython-input-24-0b91209cd92a>, line 5)


For $\alpha$ = 0.05,

$H_0$ : There is no difference in annual income between charged_off and fully_paid loans.


$H_a$ : There is a difference in annual income between charged_off and fully_paid loans.

p = 1.0

We do not reject the null hypothesis. There is no statistical difference between subscribers and customers.



In [ ]:
boostrap on the mean default rate